In [83]:
import json
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint

In [84]:
with open('data.json',encoding='utf-8') as f:
    data = json.load(f)

In [85]:
rubrics=['' for i in range(len(data))]
for i in range(len(data)):
    rubrics[i]=data[i]['page'][0]['rubric']

In [87]:
rub_uniq=(set(rubrics))
rub_uniq_2=(set(rubrics))

In [93]:
X=['' for i in range(len(data))]
for i in range(len(data)):
    X[i]=data[i]['page'][0]['stemedPlaintext']#+data[i]['page'][0]['stemedMetaDescription']

In [94]:
N=900
TF_IDF=TfidfVectorizer()
vectorize=TF_IDF.fit(X[:N])
X_train=vectorize.transform(X[:N])
X_test=vectorize.transform(X[N:])
y_train=rubrics[:N]
y_test=rubrics[N:]

In [96]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [98]:
for a in gs.grid_scores_:
    print(a.mean_validation_score,a.parameters)# — оценка качества по кросс-валидации, значения параметров

0.17777777777777778 {'C': 1e-05}
0.17777777777777778 {'C': 0.0001}
0.17777777777777778 {'C': 0.001}
0.17777777777777778 {'C': 0.01}
0.17777777777777778 {'C': 0.1}
0.6222222222222222 {'C': 1.0}
0.6511111111111111 {'C': 10.0}
0.6511111111111111 {'C': 100.0}
0.6511111111111111 {'C': 1000.0}
0.6511111111111111 {'C': 10000.0}
0.6511111111111111 {'C': 100000.0}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [99]:
k=0
for j in range(len(rub_uniq)):
    r=rub_uniq.pop()
    for i in range(len(data)):
        if rubrics[i]==r:
            rubrics[i]=k
    k+=1

In [100]:
score = 0
C = 0
for attempt in gs.grid_scores_:
    if attempt.mean_validation_score > score:
        score = attempt.mean_validation_score
        C = attempt.parameters['C']
clf = SVC(kernel='linear', random_state=241,C=C)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [101]:
clf.fit(X_train,y_train)
c=clf.predict(X_test)

In [102]:
X_train.shape, X_test.shape

((900, 21367), (100, 21367))

In [103]:
print(c[10:20])
print(y_test[10:20])

['Россия' 'Россия' 'Интернет и СМИ' 'Россия' 'Спорт' 'Мир' 'Россия'
 'Культура' 'Ценности' 'Россия']
['Силовые структуры', 'Бывший СССР', 'Интернет и СМИ', 'Россия', 'Спорт', 'Мир', 'Россия', 'Ценности', 'Ценности', 'Экономика']


In [110]:
k=0
er=[]
er_c=[]
for i in range(1000-N):
    if c[i]!=y_test[i]:
        er.append(['',''])
        er[k][0]=y_test[i]
        er[k][1]=c[i]
        er_y.append(y_test[i])
        er_c.append(c[i])
        k+=1
        print(er[k-1])
#print(set(er))

['Экономика', 'Мир']
['Силовые структуры', 'Россия']
['Силовые структуры', 'Россия']
['Бывший СССР', 'Россия']
['Ценности', 'Культура']
['Экономика', 'Россия']
['Из жизни', 'Мир']
['Путешествия', 'Мир']
['Бывший СССР', 'Мир']
['Силовые структуры', 'Россия']
['Силовые структуры', 'Россия']
['Путешествия', 'Мир']
['Интернет и СМИ', 'Мир']
['Россия', 'Силовые структуры']
['Ценности', 'Экономика']
['Крым', 'Бывший СССР']
['Россия', 'Мир']
['Из жизни', 'Мир']
['Путешествия', 'Мир']
['Силовые структуры', 'Россия']
['Силовые структуры', 'Россия']
['Силовые структуры', 'Россия']
['Россия', 'Экономика']
['Ценности', 'Из жизни']
['Бывший СССР', 'Культура']
['Силовые структуры', 'Россия']
['Экономика', 'Мир']
['', 'Россия']


In [81]:
rub_uniq_2=(set(rubrics))

In [82]:
print(rub_uniq_2)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
